## LangChain Hub

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.vectorstores import Chroma

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI

from langchain.tools.retriever import create_retriever_tool
from langchain.tools import tool

from langchain import hub

from platform import python_version

from langchain.agents import create_tool_calling_agent, AgentExecutor

In [3]:
wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

In [4]:
vectorstore = Chroma(
    persist_directory= '../vector-store-local',
    embedding_function=OpenAIEmbeddings()
    )


retriever = vectorstore.as_retriever(
    search_type = 'mmr', search_kwargs = {'k': 3, 'lambda_mult': 0.7}
)

retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="Introduction-to-Data-and-Data-Science-Course-Lectures",
    description='''For any questions regarding the
    Introduction to Data and Data Science course, you must use this tool.'''
    )






/var/folders/l0/l8bzvy196wn95cjygzkqtsfw0000gn/T/ipykernel_61588/675396890.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [5]:
@tool
def get_python_version() -> str:
    '''Useful for questions regarding the version of Python recently used.'''
    return python_version()

In [6]:
tools = [wikipedia_tool, retriever_tool, get_python_version]

In [7]:
# tools[0].name,
# tools[2].name,


In [8]:
chat = ChatOpenAI(model='gpt-4-0125-preview',
                  temperature=0,
                  model_kwargs={'seed':365})

/opt/anaconda3/envs/langchain_env_benji/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3519: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [9]:
chat_prompt_template = hub.pull('hwchase17/openai-tools-agent')

/opt/anaconda3/envs/langchain_env_benji/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [10]:
chat_prompt_template

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [11]:
chat_prompt_template.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [12]:
agent = create_tool_calling_agent(llm=chat,
                                  tools=tools,
                                  prompt=chat_prompt_template)

In [13]:
agent_executor = AgentExecutor(agent=agent,
                               tools=tools,
                               verbose=True,
                               return_intermediate_steps=True)

In [15]:
response = agent_executor.invoke({'input':"Could you tell me the version of Python I am currently using?"})



> Entering new AgentExecutor chain...

Invoking: `get_python_version` with `{}`


3.10.16The version of Python you are currently using is 3.10.16.

> Finished chain.


In [16]:
print(response)

{'input': 'Could you tell me the version of Python I am currently using?', 'output': 'The version of Python you are currently using is 3.10.16.', 'intermediate_steps': [(ToolAgentAction(tool='get_python_version', tool_input={}, log='\nInvoking: `get_python_version` with `{}`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_WdzSq4jphyasBFIZraXkCXdW', 'function': {'arguments': '{}', 'name': 'get_python_version'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4-0125-preview'}, id='run-038d5403-53f6-46c4-b3cf-08389fb38a54', tool_calls=[{'name': 'get_python_version', 'args': {}, 'id': 'call_WdzSq4jphyasBFIZraXkCXdW', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'get_python_version', 'args': '{}', 'id': 'call_WdzSq4jphyasBFIZraXkCXdW', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_WdzSq4jphyasBFIZraXkCXdW'), '3.10.16')]}


In [17]:
response2 = agent_executor.invoke({'input':'''
I'm currently following the Introduction To Data and Data science course.
                                   Could you list the programming languages a data scientst should know?
                                   Additionally, could you tell me who their creators are?'''})



> Entering new AgentExecutor chain...

Invoking: `Introduction-to-Data-and-Data-Science-Course-Lectures` with `{'query': 'programming languages for data science'}`


Thus, we need a lot of computational power, and we can expect people to use the languages similar to those in the big data column. Apart from R, Python, and MATLAB, other, faster languages are used like Java, JavaScript, C, C++, and Scala. Cool. What we said may be wonderful, but that’s not all! By using one or more programming languages, people create application software or, as they are sometimes called, software solutions, that are adjusted for specific business needs

As you can see from the infographic, R, and Python are the two most popular tools across all columns. Their biggest advantage is that they can manipulate data and are integrated within multiple data and data science software platforms. They are not just suitable for mathematical and statistical computations. In other words, R, and Python are adaptable. 

In [18]:
response2

{'input': "\nI'm currently following the Introduction To Data and Data science course.\n                                   Could you list the programming languages a data scientst should know?\n                                   Additionally, could you tell me who their creators are?",
 'output': 'Here\'s a list of programming languages a data scientist should know, along with their creators:\n\n1. **R Programming Language**: R is a language for statistical computing and graphics. It was created by Ross Ihaka and Robert Gentleman at the University of Auckland, New Zealand.\n\n2. **Python**: Python is a high-level, general-purpose programming language. It was created by Guido van Rossum and first released in 1991.\n\n3. **MATLAB**: MATLAB (Matrix Laboratory) is a multi-paradigm numerical computing environment and proprietary programming language developed by MathWorks. Cleve Moler is the chairman of MathWorks and the original author of the MATLAB software.\n\n4. **Java**: Java is a high